# Imports


In [7]:
import statistics
import tensorflow.keras.layers.experimental.preprocessing as preprocessing
import keras_tuner as kt
import numpy as np
import tensorflow.keras.layers as Layer
import tensorflow as tf
import pandas as pd


In [9]:
import re
import pandas as pd

movieFields = ['id', 'title', 'vote_average', 'vote_count', 'genres']
castFields = ['id', 'cast']

movieData = pd.read_csv('movies_metadata.csv',
                        skipinitialspace=True, usecols=movieFields, engine='python')
castData = pd.read_csv('credits.csv', skipinitialspace=True,
                       usecols=castFields, engine='python')

dfMovies = pd.DataFrame(movieData)
dfCast = pd.DataFrame(castData)

df = pd.merge(dfMovies, dfCast, how='inner')


# ---------------------------------------------------------------------------- #
# The following section formats the cast and creates a clean list of actor names.

unformattedNames = df['cast']

# main list of cast grouped by movies
cast_master_copy = []

# list of overall cast by individual names
actor_name_list = []

# reads through the cast string for each movie
for unformattedNameString in unformattedNames:

    # creates new list for current movie
    grouped = ""

    # pulls actors' names out of the paragraph
    line_list = re.findall("(?<=\'name\': )(.*?)(?=,)", unformattedNameString)

    # converts list of cast for one movie into a string to manipulate further
    line_string = str(line_list)

    # separates cast into a unique list per movie
    line_split = line_string.split(", ")

    actor_count = 0
    # reads through each actor per movie
    for name in line_split:
        if (actor_count == 5):
            break

        # removes extraneous symbols from the actors' names
        name = name.replace("\"", "")
        name = name.replace("\'", "")
        name = name.replace("[", "")
        name = name.replace("]", "")
        name = name.replace(" ", "")

        # adds actors to two working lists
        # grouped: actors are in lists by movies
        # actor_name_list: puts every actors in one list
        grouped += name + ' '
        actor_name_list.append(name)

        actor_count += 1

    # adds formatted cast members to list
    grouped = grouped[:-1]
    cast_master_copy.append(grouped)

# assigns formatted cast to 'cast' column of DataFrame
df['cast'] = cast_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section formats the genre and adds it to the data frame

unformatted_genres = df['genres']

# main list of genres grouped by movies
genre_master_copy = []

# list of overall cast by individual names
genre_name_list = []

# reads through the genre string for each movie
for unformatted_genre_string in unformatted_genres:

    # creates new list for current movie
    genre_grouped = []

    # pulls genres out of the paragraph
    genre_line_list = re.findall(
        "(?<=\'name\': )(.*?)(?=})", unformatted_genre_string)

    # converts list of genres for one movie into a string to manipulate further
    genre_line_string = str(genre_line_list)

    # separates genre into a unique list per movie
    genre_line_split = genre_line_string.split(", ")

    # reads through each genre per movie & grabs the first one
    for genre in genre_line_split:

        # removes extraneous symbols from the genres
        genre = genre.replace("\"", "")
        genre = genre.replace("\'", "")
        genre = genre.replace("[", "")
        genre = genre.replace("]", "")

        genre_str = str(genre)
        break

    # adds formatted genre to list
    genre_master_copy.append(genre_str)

# assigns formatted cast to 'cast' column of DataFrame
df['genres'] = genre_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section appends the genre to the cast string. This preparation is
# needed for text vectorization / embedding.

final_format_col = []
cast = df['cast'].to_list()
genres = df['genres'].to_list()

i = 0
count = len(cast)
for i in range(count):
    final_string = cast[i] + " " + genres[i]
    final_format_col.append(final_string)

df.insert(loc=0, column='training_col', value=final_format_col)
df.training_col = final_format_col
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section filters out data points that will be harmful for
# our model's accuracy and drops unnecessary columns.

# gets rid of all movies with 0 votes
# Number of movies before operation: 43019
# Number of movies after operation: 22651
df = df.drop(df[df.vote_count < 10].index)
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section handles the creation of the .csv files that will be the
# primary data used to train the models.

# writes formatted DataFrame values to a new csv
pd.DataFrame.to_csv(df, "formatted_movies.csv", index="false")

# creates a csv of the actor dictionary
# w = csv.writer(open("actor_dict.csv", "w"))
# for key, val in actor_dict.items():
#     w.writerow([key, val])
# ---------------------------------------------------------------------------- #

np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis=1)

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df['training_col'].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['training_col']].tail(test_data_count))

std = statistics.stdev(movie_df['vote_average'].to_list())
mean = statistics.mean(movie_df['vote_average'].to_list())
variance = statistics.variance(movie_df['vote_average'].to_list())
mode = statistics.mode(movie_df['vote_average'].to_list())


VOCAB_SIZE = 201277
encoder = preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE, output_sequence_length=6)
features = np.array(df['training_col'])
encoder.adapt(features)


FileNotFoundError: [Errno 2] No such file or directory: 'movies_metadata.csv'

# Recurrent Neural Network


In [ ]:
!rm - rf untitled_project


def build_model(hp):

    model = tf.keras.Sequential()
    model.add(encoder)

    model.add(Layer.Embedding(
              input_dim=len(encoder.get_vocabulary()),
              # output_dim=hp.Int('embedding_neurons', min_value=350, max_value=500, step=50)
              output_dim=450))

# optimizer=hp.Choice('optimizer', values=['adam', 'RMSprop'])
    optimizer = 'RMSprop'

    # activation=hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'selu'])
    activation = 'relu'

    # learning_rate = hp.Choice(
        # 'learning_rate', values=[.01, .001, .0001, .0002, .002]) 
    learning_rate=.002

    # model.add(Layer.SimpleRNN(hp.Int('rnn_neurons', min_value=150, max_value=400, step=50)))
    model.add(Layer.SimpleRNN(200))

    # for i in range(4):
    # for i in range(hp.Int('num_hidden_layers', 3, 4)):
    model.add(Layer.Dense(hp.Int(f'dense{0}_neurons', min_value=20, max_value=80, step=20), activation='relu'))
    model.add(Layer.Dense(hp.Int(f'dense{1}_neurons', min_value=20, max_value=80, step=20), activation='relu'))
    model.add(Layer.Dense(hp.Int(f'dense{2}_neurons', min_value=20, max_value=80, step=20), activation='relu'))
    # model.add(Layer.Dense(20, activation=activation))
    # model.add(Layer.Dense(80, activation=activation))
    # model.add(Layer.Dense(80, activation=activation))

    model.add(Layer.Dense(1, name='output'))

    tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=optimizer)

    return model


tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=300,  # how many model variations to test?
    executions_per_trial=3)  # how many trials per variation? (same model could perform differently

tuner.search_space_summary()


rm: -: No such file or directory
rm: rf: No such file or directory
rm: untitled_project: No such file or directory
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2021-11-24 15:14:14.052473: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-24 15:14:14.052641: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'encoder' is not defined

## RNN Training & Validation


In [ ]:
tuner.search(x=x_train,
             y=y_train,
             verbose=2,
             epochs=1,
             batch_size=512,
             callbacks=[],
             validation_data=(x_test, y_test))

tuner.results_summary()


Trial 52 Complete [00h 00m 22s]
val_loss: 1.019692301750183

Best val_loss So Far: 0.7287561297416687
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7287561297416687
Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.7335172891616821
Trial summary
Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7372773289680481
Trial summary
Hyperparameters:
embedding_neurons: 550
rnn_neurons: 250
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7380682229995728
Trial summary
Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
op

### Keras-Tuner Hyperparameter Optimization

Go in Editing Mode to View

# Keras Tuner Findings

# Test 1

general test with many values to see how the model reacts to different nums of neurons

### Search Space

Search space summary
Default search space size: 7
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 100, 'max_value': 200, 'step': 50, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 100, 'max_value': 200, 'step': 50, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
dense2_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
dense3_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 300, 'step': 50, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

### Results

Trial 300 Complete [00h 00m 07s]

Best val_loss So Far: 0.7231940627098083
Total elapsed time: 00h 30m 58s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Hyperparameters:
embedding_neurons: 150
rnn_neurons: 100
dense1_neurons: 50
dense2_neurons: 50
dense3_neurons: 50
optimizer: RMSprop
learning_rate: 0.01
Score: 0.7231940627098083

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 200
dense1_neurons: 50
dense2_neurons: 300
dense3_neurons: 150
optimizer: adam
learning_rate: 0.0001
Score: 0.7882779836654663

Hyperparameters:
embedding_neurons: 150
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 100
optimizer: adam
learning_rate: 0.0001
Score: 0.7910320162773132

Hyperparameters:
embedding_neurons: 100
rnn_neurons: 150
dense1_neurons: 50
dense2_neurons: 50
dense3_neurons: 200
optimizer: adam
learning_rate: 0.001
Score: 0.7981758117675781

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 50
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8152108192443848

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 200
dense1_neurons: 150
dense2_neurons: 250
dense3_neurons: 300
optimizer: adam
learning_rate: 0.001
Score: 0.8200659155845642

Hyperparameters:
embedding_neurons: 150
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 150
dense3_neurons: 250
optimizer: adam
learning_rate: 0.001
Score: 0.8227230906486511

Hyperparameters:
embedding_neurons: 150
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 50
dense3_neurons: 200
optimizer: adam
learning_rate: 0.001
Score: 0.8232067227363586

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 150
dense1_neurons: 100
dense2_neurons: 50
dense3_neurons: 200
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8308647871017456

Hyperparameters:
embedding_neurons: 100
rnn_neurons: 200
dense1_neurons: 200
dense2_neurons: 100
dense3_neurons: 300
optimizer: adam
learning_rate: 0.0001
Score: 0.8320332169532776

# Test 2

mainly focusing on the embedding layers for this test

### Search Space

Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 500, 'step': 100, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 500, 'step': 100, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 100, 'step': 50, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

### Results

Trial 81 Complete [00h 00m 15s]

Best val_loss So Far: 0.6759699583053589
Total elapsed time: 00h 17m 08s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Hyperparameters:
embedding_neurons: 400
rnn_neurons: 200
dense1_neurons: 50
optimizer: RMSprop
learning_rate: 0.01
Score: 0.6759699583053589

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.01
Score: 0.7111835479736328

Hyperparameters:
embedding_neurons: 200
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.001
Score: 0.7244152426719666

Hyperparameters:
embedding_neurons: 400
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.001
Score: 0.737920880317688

Hyperparameters:
embedding_neurons: 300
rnn_neurons: 300
dense1_neurons: 50
optimizer: adam
learning_rate: 0.001
Score: 0.7473349571228027

Hyperparameters:
embedding_neurons: 500
rnn_neurons: 400
dense1_neurons: 100
optimizer: adam
learning_rate: 0.0001
Score: 0.7481404542922974

Hyperparameters:
embedding_neurons: 400
rnn_neurons: 200
dense1_neurons: 50
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7580930590629578

Hyperparameters:
embedding_neurons: 300
rnn_neurons: 400
dense1_neurons: 50
optimizer: adam
learning_rate: 0.001
Score: 0.7628225088119507

Hyperparameters:
embedding_neurons: 300
rnn_neurons: 500
dense1_neurons: 50
optimizer: adam
learning_rate: 0.01
Score: 0.7678693532943726

Hyperparameters:
embedding_neurons: 400
rnn_neurons: 300
dense1_neurons: 100
optimizer: adam
learning_rate: 0.01
Score: 0.7682520151138306

# Test 3

general test with many values to see how the model reacts to different nums of neurons

### Search Space

Search space summary
Default search space size: 5
embedding_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 450, 'max_value': 550, 'step': 50, 'sampling': None}
rnn_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 200, 'max_value': 250, 'step': 50, 'sampling': None}
dense1_neurons (Int)
{'default': None, 'conditions': [], 'min_value': 60, 'max_value': 80, 'step': 20, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'RMSprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}

### Results

Trial 52 Complete [00h 00m 22s]
val_loss: 1.019692301750183

Best val_loss So Far: 0.7287561297416687
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_loss', direction='min')

Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7287561297416687

Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.7335172891616821

Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7372773289680481

Hyperparameters:
embedding_neurons: 550
rnn_neurons: 250
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.001
Score: 0.7380682229995728

Hyperparameters:
embedding_neurons: 500
rnn_neurons: 200
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8163751363754272

Hyperparameters:
embedding_neurons: 550
rnn_neurons: 200
dense1_neurons: 80
optimizer: adam
learning_rate: 0.01
Score: 0.8183265924453735

Hyperparameters:
embedding_neurons: 450
rnn_neurons: 200
dense1_neurons: 60
optimizer: adam
learning_rate: 0.01
Score: 0.8438754677772522

Hyperparameters:
embedding_neurons: 550
rnn_neurons: 200
dense1_neurons: 60
optimizer: RMSprop
learning_rate: 0.01
Score: 0.8698259592056274

Hyperparameters:
embedding_neurons: 450
rnn_neurons: 250
dense1_neurons: 80
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.8738608956336975

Hyperparameters:
embedding_neurons: 450
rnn_neurons: 250
dense1_neurons: 60
optimizer: adam
learning_rate: 0.0001
Score: 0.9030186533927917


# Tuning Embedding & RNN Trial 1 SELECTED 400, 200
Trial 21 Complete [00h 00m 22s]
Best val_loss So Far: 0.7764803171157837
Total elapsed time: 00h 04m 58s

Hyperparameters:
embedding_neurons: 400
rnn_neurons: 250
Score: 0.7764803171157837

Hyperparameters:
embedding_neurons: 350
rnn_neurons: 300
Score: 0.7991357445716858

Hyperparameters:
embedding_neurons: 250
rnn_neurons: 400
Score: 0.809856116771698


# Tuning Embedding & RNN Trial 2
Trial 16 Complete [00h 00m 23s]
Best val_loss So Far: 0.7470243573188782
Total elapsed time: 00h 04m 43s

embedding_neurons: 450
rnn_neurons: 200
Score: 0.7470243573188782

embedding_neurons: 350
rnn_neurons: 400
Score: 0.7497569918632507

embedding_neurons: 400
rnn_neurons: 300
Score: 0.7618322968482971


# Tuning Embedding & RNN Trial 3
Trial 21 Complete [00h 00m 14s]
Best val_loss So Far: 0.7542263865470886
Total elapsed time: 00h 06m 32s

embedding_neurons: 350
rnn_neurons: 300
Score: 0.7542263865470886

embedding_neurons: 450
rnn_neurons: 350
Score: 0.7763031125068665

embedding_neurons: 500
rnn_neurons: 350
Score: 0.7860965132713318














# Tuning Number of Layers Trial 1 SELECTED 3
Trial 20 Complete [00h 00m 16s]
Best val_loss So Far: 0.7369714379310608
Total elapsed time: 00h 06m 46s

Hyperparameters:
num_hidden_layers: 1
dense0_neurons: 70
dense1_neurons: 70
Score: 0.7369714379310608

Hyperparameters:
num_hidden_layers: 2
dense0_neurons: 120
dense1_neurons: 70
Score: 0.7635987401008606

Hyperparameters:
num_hidden_layers: 1
dense0_neurons: 70
dense1_neurons: 20
Score: 0.7733811736106873


# Tuning Number of Layers Trial 2
Trial 85 Complete [00h 00m 22s]
Best val_loss So Far: 0.7341732978820801
Total elapsed time: 00h 28m 27s

Hyperparameters:
num_hidden_layers: 2
dense0_neurons: 70
dense1_neurons: 70
dense2_neurons: 70
Score: 0.7341732978820801

Hyperparameters:
num_hidden_layers: 3
dense0_neurons: 70
dense1_neurons: 20
dense2_neurons: 20
Score: 0.7640082240104675

Hyperparameters:
num_hidden_layers: 3
dense0_neurons: 20
dense1_neurons: 70
dense2_neurons: 120
Score: 0.7824288606643677


# Tuning Number of Layers Trial 3
Trial 127 Complete [00h 00m 16s]
Best val_loss So Far: 0.7480059862136841
Total elapsed time: 01h 07m 08s

Hyperparameters:
dense0_neurons: 50
dense1_neurons: 50
dense2_neurons: 50
dense3_neurons: 100
Score: 0.7480059862136841

Hyperparameters:
dense0_neurons: 100
dense1_neurons: 200
dense2_neurons: 150
dense3_neurons: 50
Score: 0.773017942905426

Hyperparameters:
dense0_neurons: 200
dense1_neurons: 50
dense2_neurons: 150
dense3_neurons: 200
Score: 0.7794265151023865


# Tuning Number of Layers Trial 4
Trial 41 Complete [00h 00m 22s]
Best val_loss So Far: 0.8038003444671631
Total elapsed time: 00h 13m 56s

Hyperparameters:
dense0_neurons: 100
dense1_neurons: 150
dense2_neurons: 250
dense3_neurons: 250
Score: 0.8038003444671631

Hyperparameters:
dense0_neurons: 50
dense1_neurons: 150
dense2_neurons: 200
dense3_neurons: 200
Score: 0.8068549036979675

Hyperparameters:
dense0_neurons: 100
dense1_neurons: 200
dense2_neurons: 150
dense3_neurons: 300
Score: 0.8115206956863403


# Tuning Number of Layers Trial 5
Trial 146 Complete [00h 00m 18s]
Best val_loss So Far: 0.7369858026504517
Total elapsed time: 00h 47m 03s

Search: Running Trial #147

Hyperparameter |Value |Best Value So Far
dense0_neurons |60 |60  
dense1_neurons |20 |60  
dense2_neurons |60 |100  
dense3_neurons |40 |20


# Tuning Number of Layers Trial 6
Trial 57 Complete [00h 00m 22s]
Best val_loss So Far: 0.7318606376647949
Total elapsed time: 00h 18m 40s

Hyperparameters:
dense0_neurons: 60
dense1_neurons: 20
dense2_neurons: 80
Score: 0.7318606376647949

Hyperparameters:
dense0_neurons: 60
dense1_neurons: 20
dense2_neurons: 100
Score: 0.7404248118400574

Hyperparameters:
dense0_neurons: 20
dense1_neurons: 20
dense2_neurons: 60
Score: 0.7605499625205994


# Tuning Number of Layers Trial 7
Trial 15 Complete [00h 00m 09s]
Best val_loss So Far: 0.6739493608474731
Total elapsed time: 00h 02m 41s

Hyperparameters:
dense0_neurons: 60
dense1_neurons: 60
Score: 0.6739493608474731

Hyperparameters:
dense0_neurons: 40
dense1_neurons: 80
Score: 0.6811817288398743

Hyperparameters:
dense0_neurons: 40
dense1_neurons: 60
Score: 0.6852828860282898


# Tuning Number of Layers Trial 8
Trial 64 Complete [00h 00m 09s]
Best val_loss So Far: 0.6747193932533264
Total elapsed time: 00h 12m 02s

Hyperparameters:
dense0_neurons: 60
dense1_neurons: 20
dense2_neurons: 20
Score: 0.6747193932533264

Hyperparameters:
dense0_neurons: 40
dense1_neurons: 20
dense2_neurons: 60
Score: 0.6808950304985046

Hyperparameters:
dense0_neurons: 60
dense1_neurons: 20
dense2_neurons: 40
Score: 0.6882638335227966


# Tuning Number of Layers Trial 9
Trial 63 Complete [00h 00m 09s]
Best val_loss So Far: 0.682644248008728
Total elapsed time: 00h 11m 58s

Hyperparameters:
dense0_neurons: 50
dense1_neurons: 60
Score: 0.682644248008728

Hyperparameters:
dense0_neurons: 30
dense1_neurons: 90
Score: 0.6827664971351624

Hyperparameters:
dense0_neurons: 20
dense1_neurons: 80
Score: 0.6870942711830139

# Tuning Number of Layers Trial 10
Trial 65 Complete [00h 00m 12s]
Best val_loss So Far: 0.6783700585365295
Total elapsed time: 00h 11m 26s

Hyperparameters:
dense0_neurons: 20
dense1_neurons: 80
dense2_neurons: 80
Score: 0.6783700585365295

Hyperparameters:
dense0_neurons: 40
dense1_neurons: 20
dense2_neurons: 100
Score: 0.6829649209976196

Hyperparameters:
dense0_neurons: 80
dense1_neurons: 20
dense2_neurons: 100
Score: 0.6857151985168457









# Tuning Number of Neurons Trial 1
Best val_loss So Far: 0.6822471618652344
Total elapsed time: 00h 21m 11s
Search: Running Trial #125

Hyperparameter    |Value             |Best Value So Far 
dense0_neurons    |10                |30                
dense1_neurons    |30                |20                
dense2_neurons    |30                |30  


# Tuning Number of Neurons Trial 2
Trial 54 Complete [00h 00m 08s]
Best val_loss So Far: 0.680968701839447
Total elapsed time: 00h 09m 08s
Hyperparameters:
dense0_neurons: 20
dense1_neurons: 80
dense2_neurons: 80
Score: 0.680968701839447

Trial summary
Hyperparameters:
dense0_neurons: 40
dense1_neurons: 80
dense2_neurons: 40
Score: 0.6824204921722412

Trial summary
Hyperparameters:
dense0_neurons: 60
dense1_neurons: 20
dense2_neurons: 60
Score: 0.7183101177215576











# Tuning Optimizer Trial 1

Trial 14 Complete [00h 00m 04s]
Best val_loss So Far: 0.7041562795639038
Total elapsed time: 00h 02m 19s
Hyperparameters:
optimizer: RMSprop
learning_rate: 0.0001
Score: 0.7041562795639038

Trial summary
Hyperparameters:
optimizer: RMSprop
learning_rate: 0.001
Score: 0.789907693862915

Trial summary
Hyperparameters:
optimizer: adam
learning_rate: 0.0001
Score: 0.8452033996582031





****




# Tuning Activation Function Trial 1
Trial 4 Complete [00h 00m 12s]
Best val_loss So Far: 0.6933540105819702
Total elapsed time: 00h 00m 49s

Trial summary
activation: relu
Score: 0.6933540105819702

Trial summary
Hyperparameters:
activation: tanh
Score: 0.9309998154640198

Trial summary
Hyperparameters:
activation: selu
Score: 1.0036218166351318

Trial summary
Hyperparameters:
activation: sigmoid
Score: 5.291884422302246




 







# Tuning Batch Size & Learning Rate Trial 1
512
Trial 3 Complete [00h 00m 12s]
Best val_loss So Far: 0.6933540105819702
Total elapsed time: 00h 00m 32s

Hyperparameters:
learning_rate: 0.001
Score: 0.6933540105819702

Trial summary
Hyperparameters:
learning_rate: 0.01
Score: 0.7041562795639038

Trial summary
Hyperparameters:
learning_rate: 0.0001
Score: 0.7636291980743408


# Tuning Batch Size & Learning Rate Trial 2
512
Trial 5 Complete [00h 00m 49s]
Best val_loss So Far: 0.6814139286677042
Total elapsed time: 00h 04m 17s

Hyperparameters:
learning_rate: 0.002
Score: 0.6814139286677042

Trial summary
Hyperparameters:
learning_rate: 0.001
Score: 0.7675607204437256

Trial summary
Hyperparameters:
learning_rate: 0.0001
Score: 0.931837260723114


# Tuning Batch Size & Learning Rate Trial 3
256
Trial 5 Complete [00h 01m 07s]
Best val_loss So Far: 0.6947200496991476
Total elapsed time: 00h 05m 24s

Trial summary
Hyperparameters:
learning_rate: 0.0002
Score: 0.6947200496991476

Trial summary
Hyperparameters:
learning_rate: 0.001
Score: 0.7192908922831217

Trial summary
Hyperparameters:
learning_rate: 0.0001
Score: 0.7538880109786987


